## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\sha\anaconda3\envs\tensorflow2\lib\site-packages\ten

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


In [3]:
(x, y), (x_test, y_test)=mnist_dataset()

In [4]:
x.shape

(60000, 28, 28)

In [48]:
W1=tf.compat.v1.get_variable(shape=(784,1000),name="W1",initializer=tf.compat.v1.compat.)

In [39]:
W1

<tf.Variable 'W1:0' shape=(784, 1000) dtype=float32, numpy=
array([[-0.03016049,  0.02156356, -0.03067532, ..., -0.03337845,
         0.04598792, -0.04670276],
       [ 0.00575833,  0.00667934, -0.03433681, ..., -0.02842194,
         0.00067294, -0.03530163],
       [ 0.04034619,  0.03525447,  0.0245842 , ...,  0.03037547,
        -0.03199268,  0.03538283],
       ...,
       [ 0.04523041,  0.03374564,  0.04964223, ..., -0.05243466,
         0.00710871,  0.03805682],
       [ 0.05120077,  0.01720456,  0.05410052, ..., -0.04936979,
        -0.00783491, -0.05730088],
       [ 0.00738853,  0.01589833,  0.01087407, ...,  0.04020373,
         0.0180767 ,  0.03553743]], dtype=float32)>

## 建立模型

In [53]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1=tf.compat.v1.get_variable(shape=(784,1000),name="W1",initializer=tf.compat.v1.random_normal_initializer)
        self.b1=tf.compat.v1.get_variable(shape=(1,1000),name="b1",initializer=tf.compat.v1.random_normal_initializer)
        self.W2=tf.compat.v1.get_variable(shape=(1000,666),name="W2",initializer=tf.compat.v1.random_normal_initializer)
        self.b2=tf.compat.v1.get_variable(shape=(1,666),name="b2",initializer=tf.compat.v1.random_normal_initializer)
        
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x=tf.reshape(x,(-1,784))
        hidden = tf.nn.relu(tf.matmul(x,self.W1)+self.b1)
        logits = tf.nn.relu(tf.matmul(hidden,self.W2)+self.b2)
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [54]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [55]:
train_data, test_data = mnist_dataset()
for epoch in range(500):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    if epoch%20==0:
        print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 519.0222 ; accuracy 0.00033333333
epoch 20 : loss 185.2604 ; accuracy 0.32826668
epoch 40 : loss 149.453 ; accuracy 0.45838332
epoch 60 : loss 85.27589 ; accuracy 0.54426664
epoch 80 : loss 46.696995 ; accuracy 0.6098167
epoch 100 : loss 36.741867 ; accuracy 0.66868335
epoch 120 : loss 31.480482 ; accuracy 0.7055167
epoch 140 : loss 28.12213 ; accuracy 0.7316667
epoch 160 : loss 25.745243 ; accuracy 0.74941665
epoch 180 : loss 23.937845 ; accuracy 0.7640167
epoch 200 : loss 22.505018 ; accuracy 0.7762833
epoch 220 : loss 21.33785 ; accuracy 0.7856167
epoch 240 : loss 20.351212 ; accuracy 0.79375
epoch 260 : loss 19.505692 ; accuracy 0.8013333
epoch 280 : loss 18.767233 ; accuracy 0.8066667
epoch 300 : loss 18.111992 ; accuracy 0.8117167
epoch 320 : loss 17.526688 ; accuracy 0.81638336
epoch 340 : loss 17.000523 ; accuracy 0.8207167
epoch 360 : loss 16.52222 ; accuracy 0.82406664
epoch 380 : loss 16.082682 ; accuracy 0.82733333
epoch 400 : loss 15.678294 ; accuracy 0.8310